In [2]:
import json
from sage.all import GF, EllipticCurve, randint

# 1. Cargar resultados serializados
with open('results.json', 'r') as f:
    serializable = json.load(f)

# 2. Reconstruir curvas y generadores
curves = {}
for k_str, v in serializable.items():
    k = int(k_str)
    p = v['p']
    a = v['a']
    b = v['b']
    G_xy = tuple(v['G_xy'])
    F = GF(p)
    E = EllipticCurve(F, [a, b])
    G = E(G_xy)              # punto generador
    curves[k] = (E, G)

In [6]:
# 2. Ejecutar ECDH para cada curva y recopilar datos
ecdh_data = []
for k, (E, G) in curves.items():
    n = E.cardinality()

    # Claves privadas
    alice_priv = randint(1, n-1)
    bob_priv   = randint(1, n-1)

    # Claves públicas
    alice_pub = alice_priv * G
    bob_pub   = bob_priv   * G

    # Secreto compartido
    alice_secret = alice_priv * bob_pub
    bob_secret   = bob_priv   * alice_pub
    assert alice_secret == bob_secret

    # Mostrar en pantalla
    print(f"\n=== Curve {k}-bit ===")
    print(f"  Alice priv:      {alice_priv}")
    print(f"  Bob   priv:      {bob_priv}")
    print(f"  Shared secret:   {alice_secret}")

    # Guardar en estructura serializable
    ecdh_data.append({
        'bit_size':       k,
        'alice_priv':     int(alice_priv),
        'bob_priv':       int(bob_priv),
        'shared_secret':  [int(x_s), int(y_s)]
    })

# 3. Guardar todos los resultados en un archivo JSON
with open('ecdh_results.json', 'w') as f:
    json.dump(ecdh_data, f, indent=2)

print("\nECDH data saved to 'ecdh_results.json'")


=== Curve 32-bit ===
  Alice priv:      1191513438
  Bob   priv:      988230802
  Shared secret:   (1109668649 : 67188373 : 1)

=== Curve 40-bit ===
  Alice priv:      447432208193
  Bob   priv:      337403775726
  Shared secret:   (253590789068 : 100866656176 : 1)

=== Curve 48-bit ===
  Alice priv:      46720637724307
  Bob   priv:      174485073547161
  Shared secret:   (146209193517484 : 133188610815671 : 1)

ECDH data saved to 'ecdh_results.json'


In [8]:
import json
import hashlib

def derive_aes_key(shared_secret: int) -> bytes:
    """
    Derive a 256-bit AES key from the ECDH shared secret integer
    by hashing it with SHA-256.
    """
    # Ensure it's a Python int
    if isinstance(shared_secret, list):
        # If it's a list [x,y], take x (or combine as needed)
        shared_secret = shared_secret[0]
    # Convert secret to big-endian bytes (at least 1 byte)
    length = max(1, (shared_secret.bit_length() + 7) // 8)
    b = shared_secret.to_bytes(length, 'big')
    # Derive 32-byte key
    return hashlib.sha256(b).digest()

# 1. Leer resultados ECDH
with open('ecdh_results.json', 'r') as f:
    ecdh_data = json.load(f)

# 2. Derivar AES key para cada curva
derived = []
for entry in ecdh_data:
    k = entry['bit_size']
    shared = entry['shared_secret']
    key = derive_aes_key(shared)
    derived.append({
        'bit_size':      k,
        'alice_priv':    entry['alice_priv'],
        'bob_priv':      entry['bob_priv'],
        'shared_secret': shared,
        'aes_key_hex':   key.hex()
    })
    print(f"{k}-bit curve → AES key: {key.hex()}")

# 3. Guardar claves derivadas
with open('derived_keys.json', 'w') as f:
    json.dump(derived, f, indent=2)

print("\nAES keys saved to 'derived_keys.json'")


32-bit curve → AES key: 0111409ecbb111550f23de3b1eec810ede39d75f03710e29296dbb03396c7cbe
40-bit curve → AES key: 0111409ecbb111550f23de3b1eec810ede39d75f03710e29296dbb03396c7cbe
48-bit curve → AES key: 0111409ecbb111550f23de3b1eec810ede39d75f03710e29296dbb03396c7cbe

AES keys saved to 'derived_keys.json'


In [21]:
import json
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes

# 1. Leer la clave derivada para la curva de 32 bits
with open('derived_keys.json', 'r') as f:
    derived = json.load(f)

# Buscar la entrada de 32 bits
key_hex = next(entry['aes_key_hex'] for entry in derived if entry['bit_size'] == 32)
key = bytes.fromhex(key_hex)

# 2. Mensaje a cifrar: cita histórica de Carl Friedrich Gauss
# Cita: “Mathematics is the queen of the sciences and number theory is the queen of mathematics.”
# Fuente: Carl Friedrich Gauss, Disquisitiones Arithmeticae (1801)
plaintext = (
    "Mathematics is the queen of the sciences and number theory is the queen of mathematics. Carl Friedrich Gauss"
).encode('utf-8')

# 3. Cifrar con AES-256-CBC y PKCS#7 padding
iv = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_CBC, iv)
ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))

# 4. Mostrar resultados en Base64
print("IV (Base64):        ", base64.b64encode(iv).decode('utf-8'))
print("Ciphertext (Base64):", base64.b64encode(ciphertext).decode('utf-8'))

# 5. Guardar en archivo
with open('cipher_quote_gauss.txt', 'w') as f:
    f.write(f"IV:{base64.b64encode(iv).decode()}\n")
    f.write(f"CIPHER:{base64.b64encode(ciphertext).decode()}\n")


IV (Base64):         89PuzxknoQXGUEHByJ9wlg==
Ciphertext (Base64): TUQzcBuCTpRmYzX3ZKOax3+eUnN6Knuv8xXeLZ9vDuoYwLtUgFK7Fp1jzUt80I8u/NMhnFcLTMEyS7F2ue/ODqLSwlfV2xC/rbG9hHOdO0c/sHi0RWI8nSS17VaTLPvmv74IVzC+zQR659R03363jg==


In [24]:
from Crypto.Util.Padding import unpad
ciphertext = base64.b64decode("TUQzcBuCTpRmYzX3ZKOax3+eUnN6Knuv8xXeLZ9vDuoYwLtUgFK7Fp1jzUt80I8u/NMhnFcLTMEyS7F2ue/ODqLSwlfV2xC/rbG9hHOdO0c/sHi0RWI8nSS17VaTLPvmv74IVzC+zQR659R03363jg==")

# 3. Decrypt
cipher = AES.new(key, AES.MODE_CBC, iv)
decrypted = cipher.decrypt(ciphertext)

# 4. Output the result
print("Decrypted text:", decrypted.decode('utf-8'))


Decrypted text: Mathematics is the queen of the sciences and number theory is the queen of mathematics. Carl Friedrich Gauss
